In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
%matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.linalg as la
import pyvista as pv
import imageio.v3 as iio
np.set_printoptions(formatter={"float": "{:10.2f}".format})

In [3]:
from src.experiment import Experiment
from src.pycv.dmcp import dm_to_world
mesh_path = "/home/julian/code/uni/ushichka-registration/data/formatted/lidar_roi.ply"
pvMesh = pv.read(mesh_path)

In [4]:
day = "/home/julian/data/dmcp_experiments/round_03/2018-08-19"
def path_day_cam(day,cam):
    exp_path = f"{day}--cam{cam}"
    return exp_path

In [5]:
pc0 = path_day_cam(day,0)
pc1 = path_day_cam(day,1)
pc2 = path_day_cam(day,2)

exp0 = Experiment(pc0,mesh_path)
exp1 = Experiment(pc1,mesh_path)
exp2 = Experiment(pc2,mesh_path)
exps = [exp0, exp1, exp2]

In [6]:
exp1.visualize_3D()

[      2.47      -3.15      -0.75]


In [7]:
imP = exp0.load_imP()
tra = exp0.load_transform()

pos_trans = imP @ la.inv(tra)
pos_trans = (la.null_space(pos_trans) / la.null_space(pos_trans)[-1])[:3].flatten()
pos_sph_P = pv.Sphere(radius=0.25, center=pos_trans)


pos_nat = (la.null_space(imP) / la.null_space(imP)[-1])[:3].flatten()
pos_trans = tra @ np.append(pos_nat, [1])
pos_sph_pos = pv.Sphere(radius=0.25, center=pos_trans[:3])

imP1 = exp1.load_imP()
pos_nat = (la.null_space(imP1) / la.null_space(imP1)[-1])[:3].flatten()
tra[:3,:3] = tra[:3,:3]
pos_trans = tra @ np.append(pos_nat, [1])

pos_sph_pos1 = pv.Sphere(radius=0.25, center=pos_trans[:3])
print(pos_trans)

pos_trans = imP1 @ la.inv(tra)
pos_trans = (la.null_space(pos_trans) / la.null_space(pos_trans)[-1])[:3].flatten()
pos_sph_P1 = pv.Sphere(radius=0.25, center=pos_trans)
print(pos_trans)

pl = pv.Plotter(notebook=False)
pl.add_mesh(pvMesh,color="brown")
pl.add_mesh(pos_sph_P, color="green") 
pl.add_mesh(pos_sph_pos, color="blue") 
pl.add_mesh(pos_sph_pos1, color="orange")   
pl.add_mesh(pos_sph_P1, color="red") 

pl.show()

[      0.38       2.42       0.00       1.00]
[      0.38       2.42       0.00]


In [9]:
dmIm = exp0.load_dmIm()
dmK = exp0.load_dmK()
dmP = exp0.load_dmP()
cps = exp0.load_cps()
imK = exp0.load_imK()
imP = exp0.load_imP()
from src.pycv.dmcp import dmcp
pts_world = dm_to_world(dmIm, dmK, dmP, cps[:,2:])
        
print("executing sparse correspondence alignment (SCA)")

_raw_pose, trans = dmcp(imK, imP, cps[:,:2], pts_world,return_raw_pose=True)
print(f"transformation\n{trans}")

executing sparse correspondence alignment (SCA)
estimated pose
[[      0.20      -0.12      -0.97      -0.73]
 [      0.98      -0.04       0.20       3.35]
 [     -0.06      -0.99       0.11       0.53]]
scale factor 0.39107712441268583
transformation
[[     -0.26       0.27      -0.11      -1.81]
 [     -0.29      -0.27       0.01       3.10]
 [      0.07      -0.08      -0.38       0.45]
 [      0.00       0.00       0.00       1.00]]


In [10]:

box_world = pts_world
box_native_x_native = cps[:,:2]
K_native = imK
P_native = imP
from src.pycv.perspective import solve_PnP
import cv2

# fix scaling 
def fix_scaling(K,P):
    extrinsic_matrix_native = np.matmul(la.inv(K), P) 
    E1 = extrinsic_matrix_native
    sv = [la.norm(E1[0:3, 0]), la.norm(E1[0:3, 1]), la.norm(E1[0:3, 2])]
    E1[0:3,0] = E1[0:3,0] / sv[0]
    E1[0:3,1] = E1[0:3,1] / sv[1]
    E1[0:3,2] = E1[0:3,2] / sv[2]
    P_fixed = K @ E1
    return P_fixed

P_native = fix_scaling(K_native, P_native)

extrinsic_matrix_native = np.matmul(la.inv(K_native), P_native) 
E1 = extrinsic_matrix_native
sv = [la.norm(E1[0:3, 0]), la.norm(E1[0:3, 1]), la.norm(E1[0:3, 2])]
print("scale\n",sv)

scale
 [0.9999999999999999, 1.0, 1.0]


In [12]:
imP1 = exp0.load_imP()
pose_nat = (la.null_space(imP1) / la.null_space(imP1)[-1])[:3].flatten()

imP1_world = imP1 @ la.inv(exp0.load_transform())
pose_world_true = (la.null_space(imP1_world) / la.null_space(imP1_world)[-1])[:3].flatten()


print(pose_nat, "\n",pose_world_true)

[     -2.27       1.48      -0.94] 
 [     -0.72       3.35       0.53]


In [14]:
pose_matrix = la.inv(exp0.load_imK()) @ exp0.load_imP()
pose_matrix_hat = np.vstack((pose_matrix, [0,0,0,1]))
extrinsic_matrix_world = la.inv(pose_matrix_hat)[:3,:]
box_world = np.vstack((pts_world, pose_world_true))
box_world_hat = np.hstack((box_world, np.ones((box_world.shape[0],1))))
box_world_camera = np.matmul(extrinsic_matrix_world, box_world_hat.T).T
# DMCP Step 2.2 transform camera points into native space
extrinsic_matrix_native = np.matmul(la.inv(K_native), P_native) 
extrinsic_matrix_native_hat = np.vstack((extrinsic_matrix_native,[0,0,0,1]))
pose_matrix = la.inv(extrinsic_matrix_native_hat)
camera_pose_matrix_native = pose_matrix

box_world_camera_hat = np.hstack((box_world_camera,np.ones((box_world_camera.shape[0],1))))
box_native_tf = np.matmul(camera_pose_matrix_native, box_world_camera_hat.T).T[:,:3]
print(f"box_world\n{box_world}\nbox camera\n{box_world_camera}")
# DMCP Step 2.3
#A_tf = horn_affine_transformation(box_native_tf, box_world)
retval, scale = cv2.estimateAffine3D(box_native_tf, box_world,force_rotation=False)
print(f"est scale {scale}")
A_tf = retval
A_tf[:3,:3] = A_tf[:3,:3] * scale
A_tf_hat = np.vstack((A_tf,[0,0,0,1]))

box_world
[[     -3.18       5.12       0.70]
 [     -4.13       5.14       0.89]
 [     -4.44       4.91       1.42]
 [     -7.71       3.17       1.76]
 [     -0.72       3.35       0.53]]
box camera
[[      4.78       6.16      12.05]
 [      7.10       7.02      12.12]
 [      8.53       6.18      11.75]
 [     16.40       9.06       7.28]
 [     -0.54       2.62       7.68]]
est scale 0.3914386930246882


# observation of ushichka camera space does not work, even with fixed scaling

In [98]:
box_native_hat = np.hstack((box_native_tf, np.ones((box_native_tf.shape[0],1))))
transformed_native = (A_tf @ box_native_hat.T).T

print(f"box world\n{box_world}\ntrans native\n{transformed_native}")

box world
[[     -3.18       5.12       0.70]
 [     -4.13       5.14       0.89]
 [     -4.44       4.91       1.42]
 [     -7.71       3.17       1.76]
 [      2.47      -3.15      -0.75]]
trans native
[[     -3.18       5.12       0.70]
 [     -4.13       5.14       0.89]
 [     -4.44       4.91       1.42]
 [     -7.71       3.17       1.76]
 [      2.47      -3.15      -0.75]]
